# Import

In [70]:
!pip uninstall lingua -y
!pip uninstall lingua-language-detector -y
!pip install lingua-language-detector
!pip show lingua-language-detector
!pip install deep_translator

Found existing installation: lingua-language-detector 2.0.2
Uninstalling lingua-language-detector-2.0.2:
  Successfully uninstalled lingua-language-detector-2.0.2
  Using cached lingua_language_detector-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (349 kB)
Using cached lingua_language_detector-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (74.7 MB)
Name: lingua-language-detector
Version: 2.0.2
Summary: An accurate natural language detection library, suitable for short text and mixed-language text
Home-page: https://github.com/pemistahl/lingua-rs
Author: Peter M. Stahl <pemistahl@gmail.com>
Author-email: "Peter M. Stahl" <pemistahl@gmail.com>
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


# raw contents가 하면 valid하면 citation summarizing는 SKIP하고 raw contents 반환하는 것으로 수정

## GPT4o Base Model

In [78]:
import requests
import re
from lingua import LanguageDetectorBuilder, Language
from deep_translator import GoogleTranslator

# Azure OpenAI 및 Azure Search 설정
AZURE_OPENAI_ENDPOINT = 'https://openai-team3.openai.azure.com/'
AZURE_OPENAI_API_KEY = '7u1gKFj9UhcAVQD1lLfCIrlXGVWm894NEljVILS0jG7z95cCwSkHJQQJ99ALACHYHv6XJ3w3AAABACOGKTgQ'
DEPLOYMENT_NAME = 'gpt-4o'
SEARCH_ENDPOINT = "https://ai-search-team3.search.windows.net"
SEARCH_API_KEY = "SVteEFiHV62vyfZ56a6xsfba10kfSLoGseuenSigufAzSeCa4IB1"
SEARCH_INDEX_NAME = "immigrationlaw-index"
SEMANTIC_CONFIGURATION = "immigrationlaw-semantic"

# 언어 감지기 초기화
detector = LanguageDetectorBuilder.from_all_languages().build()

def detect_language(text):
    """텍스트의 언어를 감지"""
    detected_language = detector.detect_language_of(text)
    if detected_language is not None:
        return detected_language.name.lower()  # 언어 이름을 소문자로 반환
    else:
        return "unknown"

def translate_to_korean(text):
    """사용자 입력을 한국어로 번역"""
    try:
        return GoogleTranslator(source='auto', target='ko').translate(text)
    except Exception as e:
        print(f"Error in translation: {e}")
        return text  # 번역 실패 시 원본 텍스트 반환

def clean_response(text):
    """불필요한 기호 및 문구를 정리하는 함수"""
    cleaned_text = re.sub(r'\[doc\d+\]', '', text)
    cleaned_text = re.sub(r'[\[\]]', '', cleaned_text)
    return cleaned_text.strip()

def validate_citations(citations):
    """
    Citations 내용이 비어있거나 너무 짧은 경우 False 반환.
    """
    print(f"\nValidating citations... Total citations: {len(citations)}")
    if not citations:
        print("\nNo citations found.")
        return False
    for i, citation in enumerate(citations):
        content = citation.get('content', '')
        print(f"\nCitation {i + 1}: {content[:50]}...")  # 첫 50자만 출력
        if len(content.strip()) < 1:  # 너무 짧은 내용은 무효 처리
            print("\nInvalid citation detected (too short).")
            return False
    return True

def summarize_with_gpt(citations, user_input, target_language):
    """Citation 데이터를 요약하여 질문에 답변 생성"""
    print("\nSummarizing citations using GPT...")
    combined_text = " ".join([citation.get('content', '').strip() for citation in citations])

    summarization_prompt = f"""
    Question: {user_input}
    Summarize the following information in {target_language} to answer the question:
    {combined_text}
    """

    payload = {"messages": [{"role": "user", "content": summarization_prompt}], "temperature": 0.7, "max_tokens": 1000}
    headers = {"Content-Type": "application/json", "api-key": AZURE_OPENAI_API_KEY}

    response = requests.post(
        f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",
        headers=headers, json=payload
    )

    if response.status_code == 200:
        return clean_response(response.json()['choices'][0]['message']['content'].strip())
    else:
        print(f"\nError during summarization: {response.status_code}")
        return "An error occurred while summarizing the citations."

def fallback_to_gpt(question, language):
    """Search 실패 시 GPT 자체 지식으로 답변 생성"""
    print("Executing fallback: Generating response using GPT knowledge.")
    fallback_prompt = f"""
    Question: {question}
    Answer (in {language}): If the question is related to Korean law, provide a detailed and accurate response based on general legal knowledge specific to Korean law.

    At the end of your legal response, include this message translated into {language}:
    'Please refer to trusted legal sources to verify the most up-to-date information.'

    If the question is not related to law (e.g., greetings, insults, unrelated content), respond with:
    'I can only answer questions related to Korean law. Please ask questions about Korean legal matters.' in {language}
    """
    payload = {"messages": [{"role": "user", "content": fallback_prompt}], "temperature": 0.7, "max_tokens": 1000}

    headers = {"Content-Type": "application/json", "api-key": AZURE_OPENAI_API_KEY}
    response = requests.post(
        f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",
        headers=headers,
        json=payload
    )
    if response.status_code == 200:
        return clean_response(response.json()['choices'][0]['message']['content'].strip())
    else:
        return "An error occurred while generating the fallback response."

def ask_legal_question(user_input):
    """질문에 대한 응답 반환"""
    print(f"\nProcessing question: {user_input}")

    # 언어 감지 및 번역
    detected_language = detect_language(user_input)
    print(f"\nDetected language: {detected_language}")
    translated_question = translate_to_korean(user_input)
    print(f"\nTranslated question for search: {translated_question}")

    headers = {"Content-Type": "application/json", "api-key": AZURE_OPENAI_API_KEY}
    prompt = f"Question: {translated_question}\nAnswer: Provide an answer using retrieved documents in Korean."

    payload = {
        "messages": [
            {"role": "system", "content": "You are a multilingual assistant specializing in Korean law. Answer all questions about Korean law in the language of the question."},
            {"role": "user", "content": prompt}
        ],
        "data_sources": [
            {"type": "azure_search",
             "parameters": {
                 "endpoint": SEARCH_ENDPOINT,
                 "index_name": SEARCH_INDEX_NAME,
                 "semantic_configuration": SEMANTIC_CONFIGURATION,
                 "query_type": "semantic",
                 "top_n_documents": 20,
                 "authentication": {"type": "api_key", "key": SEARCH_API_KEY}
             }}
        ],
        "temperature": 0.7, "max_tokens": 800, "top_p": 0.95
    }

    try:
        print("\nSending request to Azure OpenAI...")
        response = requests.post(
            f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",
            headers=headers, json=payload
        )
        print("response.status_code: ", response.status_code)
        if response.status_code != 200:
            raise Exception(f"Azure Search request failed with status code {response.status_code}")

        result = response.json()
        raw_content = result['choices'][0]['message']['content'].strip()
        print(f"raw_content: {raw_content}")
        citations = result['choices'][0]['message'].get('context', {}).get('citations', [])

        # Citations 내용 검사
        if validate_citations(citations):
            print("\nValid citations found.")
            if "The requested information is not available" in raw_content or not raw_content:
                print("Raw content is insufficient. Using citations for the response.")
                return summarize_with_gpt(citations, user_input, detected_language)
            else:
                print("\nReturning raw_content as it is valid and sufficient.")
                return clean_response(raw_content)  # raw_content를 그대로 반환
        else:
            print("\nNo valid citations found. Triggering fallback...")
            return fallback_to_gpt(user_input, detected_language)

    except Exception as e:
        print(f"Error encountered: {e}")
        return fallback_to_gpt(user_input, detected_language)


In [79]:
ask_legal_question("나는 난민인데 대한민국 출입국 시 제출이 필요한 서류가 뭐가 있어?")


Processing question: 나는 난민인데 대한민국 출입국 시 제출이 필요한 서류가 뭐가 있어?

Detected language: korean

Translated question for search: 나는 난민인데 대한민국 출입국 시 제출이 필요한 서류가 뭐가 있어?

Sending request to Azure OpenAI...
response.status_code:  200
raw_content: 대한민국 출입국 시 난민 신청자에게 필요한 서류는 다음과 같습니다:

1. 난민 임시상륙허가 신청서: 난민 임시상륙허가를 신청할 때에는 그 이유를 설명하는 서류를 첨부하여야 하며, 이를 출입국관리공무원에게 제출해야 합니다 [doc1].

2. 난민증명서: 난민 신청자는 난민증명서를 발급받아야 하며, 이를 출입국 시 제시해야 할 수 있습니다 [doc6].

3. 여권 또는 여행증명서: 출입국 시 본인의 신원을 증명할 수 있는 여권 또는 여행증명서를 제출해야 합니다 [doc5].

이 외에도 개별 상황에 따라 추가 서류가 필요할 수 있으므로, 출입국 전에 관련 기관에 문의하여 정확한 정보를 확인하는 것이 좋습니다.

Validating citations... Total citations: 20

Citation 1: 외국인의 입국 및 상륙 - 제2절 외국인의 상륙
1 법 제16조의2제1항에 따라 난민 임시...

Citation 2: 외국인의 입국 및 상륙 &lt;개정 2010. 5. 14.&gt; - 제2절 외국인의 상륙...

Citation 3: 
1 영 제23조에 따라 난민지원시설을 이용하려는 사람은 별지 제24호의 난민지원시 설 이...

Citation 4: 강제퇴거등 - 제1절 조사
1 출입국관리공무원은 법 제50조 및 이 영 제61조에 따라 서...

Citation 5: 
1 영 제23조에 따른 국적 판정 신청서는 별지 제 11호서식에 따른다.
2 제1항의 국...

Citation 6: 난민인정 신청과 심사 등
난민심사관은 난민신청자가

'대한민국 출입국 시 난민 신청자에게 필요한 서류는 다음과 같습니다:\n\n1. 난민 임시상륙허가 신청서: 난민 임시상륙허가를 신청할 때에는 그 이유를 설명하는 서류를 첨부하여야 하며, 이를 출입국관리공무원에게 제출해야 합니다 .\n\n2. 난민증명서: 난민 신청자는 난민증명서를 발급받아야 하며, 이를 출입국 시 제시해야 할 수 있습니다 .\n\n3. 여권 또는 여행증명서: 출입국 시 본인의 신원을 증명할 수 있는 여권 또는 여행증명서를 제출해야 합니다 .\n\n이 외에도 개별 상황에 따라 추가 서류가 필요할 수 있으므로, 출입국 전에 관련 기관에 문의하여 정확한 정보를 확인하는 것이 좋습니다.'

## GPT4o fine-tune Model

In [102]:
import requests
import re
from lingua import LanguageDetectorBuilder, Language
from deep_translator import GoogleTranslator

# Azure OpenAI 및 Azure Search 설정
AZURE_OPENAI_ENDPOINT = 'https://team3openaieastus2.openai.azure.com/'
AZURE_OPENAI_API_KEY = 'F0Ubo0cRW3pGEc6fMufG6ZqujkZMzN9ZXCEL4ecYIoEIZ7SgijeiJQQJ99ALACHYHv6XJ3w3AAABACOGcf0q'
DEPLOYMENT_NAME = 'gpt-4o-fiine-tune-1m1qna'
SEARCH_ENDPOINT = "https://ai-search-team3.search.windows.net"
SEARCH_API_KEY = "SVteEFiHV62vyfZ56a6xsfba10kfSLoGseuenSigufAzSeCa4IB1"
SEARCH_INDEX_NAME = "immigrationlaw-index"
SEMANTIC_CONFIGURATION = "immigrationlaw-semantic"

# 언어 감지기 초기화
detector = LanguageDetectorBuilder.from_all_languages().build()

def detect_language(text):
    """텍스트의 언어를 감지"""
    detected_language = detector.detect_language_of(text)
    if detected_language is not None:
        return detected_language.name.lower()  # 언어 이름을 소문자로 반환
    else:
        return "unknown"

def translate_to_korean(text):
    """사용자 입력을 한국어로 번역"""
    try:
        return GoogleTranslator(source='auto', target='ko').translate(text)
    except Exception as e:
        print(f"Error in translation: {e}")
        return text  # 번역 실패 시 원본 텍스트 반환

def clean_response(text):
    """불필요한 기호 및 문구를 정리하는 함수"""
    cleaned_text = re.sub(r'\[doc\d+\]', '', text)
    cleaned_text = re.sub(r'[\[\]]', '', cleaned_text)
    return cleaned_text.strip()

def validate_citations(citations):
    """
    Citations 내용이 비어있거나 너무 짧은 경우 False 반환.
    """
    print(f"\nValidating citations... Total citations: {len(citations)}")
    if not citations:
        print("\nNo citations found.")
        return False
    for i, citation in enumerate(citations):
        content = citation.get('content', '')
        print(f"\nCitation {i + 1}: {content[:50]}...")  # 첫 50자만 출력
        if len(content.strip()) < 1:  # 너무 짧은 내용은 무효 처리
            print("\nInvalid citation detected (too short).")
            return False
    return True

def summarize_with_gpt(citations, user_input, target_language):
    """Citation 데이터를 요약하여 질문에 답변 생성"""
    print("\nSummarizing citations using GPT...")
    combined_text = " ".join([citation.get('content', '').strip() for citation in citations])

    summarization_prompt = f"""
    Question: {user_input}
    Summarize the following information in {target_language} to answer the question:
    {combined_text}
    """

    payload = {"messages": [{"role": "user", "content": summarization_prompt}], "temperature": 0.7, "max_tokens": 1000}
    headers = {"Content-Type": "application/json", "api-key": AZURE_OPENAI_API_KEY}

    response = requests.post(
        f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",
        headers=headers, json=payload
    )

    if response.status_code == 200:
        return clean_response(response.json()['choices'][0]['message']['content'].strip())
    else:
        print(f"\nError during summarization: {response.status_code}")
        return "An error occurred while summarizing the citations."

def fallback_to_gpt(question, language):
    """Search 실패 시 GPT 자체 지식으로 답변 생성"""
    print("Executing fallback: Generating response using GPT knowledge.")
    fallback_prompt = f"""
    Question: {question}
    Answer (in {language}): If the question is related to Korean law, provide a detailed and accurate response based on general legal knowledge specific to Korean law.

    At the end of your legal response, include this message translated into {language}:
    'Please refer to trusted legal sources to verify the most up-to-date information.'

    If the question is not related to law (e.g., greetings, insults, unrelated content), respond with:
    'I can only answer questions related to Korean law. Please ask questions about Korean legal matters.' in {language}
    """
    payload = {"messages": [{"role": "user", "content": fallback_prompt}], "temperature": 0.7, "max_tokens": 1000}

    headers = {"Content-Type": "application/json", "api-key": AZURE_OPENAI_API_KEY}
    response = requests.post(
        f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",
        headers=headers,
        json=payload
    )
    if response.status_code == 200:
        return clean_response(response.json()['choices'][0]['message']['content'].strip())
    else:
        return "An error occurred while generating the fallback response."

def ask_legal_question(user_input):
    """질문에 대한 응답 반환"""
    print(f"\nProcessing question: {user_input}")

    # 언어 감지 및 번역
    detected_language = detect_language(user_input)
    print(f"\nDetected language: {detected_language}")
    translated_question = translate_to_korean(user_input)
    print(f"\nTranslated question for search: {translated_question}")

    headers = {"Content-Type": "application/json", "api-key": AZURE_OPENAI_API_KEY}
    prompt = f"Question: {translated_question}\nAnswer: Provide a detailed answer using retrieved documents in Korean."

    payload = {
        "messages": [
            {"role": "system", "content": "You are a multilingual assistant specializing in Korean law. Answer all questions about Korean law in the language of the question."},
            {"role": "user", "content": prompt}
        ],
        "data_sources": [
            {"type": "azure_search",
             "parameters": {
                 "endpoint": SEARCH_ENDPOINT,
                 "index_name": SEARCH_INDEX_NAME,
                 "semantic_configuration": SEMANTIC_CONFIGURATION,
                 "query_type": "semantic",
                 "top_n_documents": 20,
                 "authentication": {"type": "api_key", "key": SEARCH_API_KEY}
             }}
        ],
        "temperature": 0.7, "max_tokens": 2000, "top_p": 0.95
    }

    try:
        print("\nSending request to Azure OpenAI...")
        response = requests.post(
            f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",
            headers=headers, json=payload
        )
        print("response.status_code: ", response.status_code)
        if response.status_code != 200:
            raise Exception(f"Azure Search request failed with status code {response.status_code}")

        result = response.json()
        raw_content = result['choices'][0]['message']['content'].strip()
        print(f"raw_content: {raw_content}")
        citations = result['choices'][0]['message'].get('context', {}).get('citations', [])

        # Citations 내용 검사
        if validate_citations(citations):
            print("\nValid citations found.")
            if "The requested information is not available" in raw_content or not raw_content:
                print("Raw content is insufficient. Using citations for the response.")
                return summarize_with_gpt(citations, user_input, detected_language)
            else:
                print("\nReturning raw_content as it is valid and sufficient.")
                return clean_response(raw_content)  # raw_content를 그대로 반환
        else:
            print("\nNo valid citations found. Triggering fallback...")
            return fallback_to_gpt(user_input, detected_language)

    except Exception as e:
        print(f"Error encountered: {e}")
        return fallback_to_gpt(user_input, detected_language)


In [103]:
ask_legal_question("나는 난민인데 대한민국 출입국 시 제출이 필요한 서류가 뭐가 있어?")


Processing question: 나는 난민인데 대한민국 출입국 시 제출이 필요한 서류가 뭐가 있어?

Detected language: korean

Translated question for search: 나는 난민인데 대한민국 출입국 시 제출이 필요한 서류가 뭐가 있어?

Sending request to Azure OpenAI...
response.status_code:  200
raw_content: 대한민국에 입국하려는 난민은 출입국 허가를 받기 위해 관련 서류를 제출해야 합니다. 출입국 허가를 신청하는 사람은 법무부령으로 정하는 서류를 첨부하여 재외공관의 장, 청장, 사무소장 또는 출장소장에게 제출해야 합니다 [doc6].

Validating citations... Total citations: 9

Citation 1: 외교(A-1) 또는 2..공무(A-2) 체류자격과 체류기간을 적어야 한다. &lt; 개정 ...

Citation 2: 외교(A-1)부터 3..협정(A-3)까지 또는 체류자격 11....

Citation 3: 외교(A-1)부터 3..협정(A-3)까지에 해 당하는 체류자격과 체류기간을 적어야 한다. ...

Citation 4: 외국인의 입국 및 상륙 - 제1절 외국인의 입국
1 법무부장관은 법 제8조제2항에 따라 별...

Citation 5: 외국인의 입국 및 상륙 - 제1절 외국인의 입국
1단수사증의 유효기간은 발급일부터 3개월로...

Citation 6: 외국인의 입국 및 상륙 - 제1절 외국인의 입국
1 법무부장관은 법 제7조제4항에 따라 외...

Citation 7: 외국인의 등록 및 사회통합 프로그램 &lt;개정 2012. 10. 15.&gt; - 제1절...

Citation 8: 외국인의 체류와 출국 - 제2절 외국인의 출국
1 법 제30조제1항 단서에서 "법무부령으로...

Citation 9: 외국인의 입국 및 상륙 - 제1절 외국인의 입국
1 법 제12조의2제1항제3호에서 "대통령...

Valid

'대한민국에 입국하려는 난민은 출입국 허가를 받기 위해 관련 서류를 제출해야 합니다. 출입국 허가를 신청하는 사람은 법무부령으로 정하는 서류를 첨부하여 재외공관의 장, 청장, 사무소장 또는 출장소장에게 제출해야 합니다 .'